# Toxic Comment Classification

In [11]:
import pandas as pd
import os


In [12]:
file = "train.csv/train.csv"
train = pd.read_csv(file)


In [13]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             159571 non-null  object
 1   comment_text   159571 non-null  object
 2   toxic          159571 non-null  int64 
 3   severe_toxic   159571 non-null  int64 
 4   obscene        159571 non-null  int64 
 5   threat         159571 non-null  int64 
 6   insult         159571 non-null  int64 
 7   identity_hate  159571 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 9.7+ MB


In [15]:
train.shape

(159571, 8)

In [20]:
train.toxic.value_counts()


0    144277
1     15294
Name: toxic, dtype: int64

In [19]:
train.severe_toxic.value_counts()

0    157976
1      1595
Name: severe_toxic, dtype: int64

In [21]:
train.obscene.value_counts()

0    151122
1      8449
Name: obscene, dtype: int64

In [22]:
train.threat.value_counts()

0    159093
1       478
Name: threat, dtype: int64

In [23]:
train.insult.value_counts()

0    151694
1      7877
Name: insult, dtype: int64

In [24]:
train.identity_hate.value_counts()

0    158166
1      1405
Name: identity_hate, dtype: int64